In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from tqdm.auto import tqdm
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
import plotly.graph_objects as go
import plotly.express as px
import joblib
from tqdm.notebook import tqdm
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import ydata_profiling
except ImportError:
    install('ydata_profiling')

try:
    from PIL import Image
except ImportError:
    install('Pillow')

try:
    import shap
except ImportError:
    install('shap')

try:
    from PIL import Image
    installed_version = Image.__version__
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "Pillow"])
except ImportError:
    install('Pillow')

import shap
from ydata_profiling import ProfileReport

pd.set_option('display.max_rows', None)

# Data Loading, Pre-Processing and Dataset Building

## Data Cleaning & Merging

In [ ]:
# Loading the data
medals_per_country = pd.read_csv('/content/medals_total.csv')
world_population = pd.read_excel('/content/world_population.xlsx')
athletes_per_country = pd.read_excel('/content/Athletes.xlsx')

world_population.rename(columns = {'Region/Country': 'Country'}, inplace = True)
hdi.columns = hdi.columns.str.strip()
hdi['Country'].replace({'Korea (Republic of)': 'Republic of Korea', 'Moldova (Republic of)': 'Republic of Moldova'}, inplace = True)

# Remove leading/trailing spaces and convert to uppercase
world_population['ISO3 Alpha-code'] = world_population['ISO3 Alpha-code'].str.strip().str.upper()

# Retrieving unique ISO3 codes
world_population_iso3_codes = world_population['ISO3 Alpha-code'].unique()

# Retrieving unique country codes
medals_country_codes = medals_per_country['country_code'].unique()

differences = set(medals_country_codes) - set(world_population_iso3_codes)
print(differences)

# Mapping country codes to ISO3 alpha codes
manual_mapping = {
    'MGL': 'MNG',  # Mongolia
    'FIJ': 'FJI',  # Fiji
    'RSA': 'ZAF',  # South Africa
    'GER': 'DEU',  # Germany
    'KOS': 'XKX',  # Kosovo
    'CRO': 'HRV',  # Croatia
    'SUI': 'CHE',  # Switzerland
    'SLO': 'SVN',  # Slovenia
    'GUA': 'GTM',  # Guatemala
    'GRE': 'GRC',  # Greece
    'NED': 'NLD',  # Netherlands
    'POR': 'PRT',  # Portugal
    'AIN': 'Individual Neutral Athletes',
    'PHI': 'PHL',  # Philippines
    'GRN': 'GRD',  # Grenada
    'TPE': 'TWN',  # Taiwan (Chinese Taipei)
    'DEN': 'DNK',  # Denmark
    'ALG': 'DZA',  # Algeria
    'MAS': 'MYS',  # Malaysia
    'INA': 'IDN',  # Indonesia
    'CHI': 'CHL',  # Chile
    'IRI': 'IRN',  # Iran
    'ZAM': 'ZMB',  # Zambia
    'BOT': 'BWA',  # Botswana
    'BUL': 'BGR',  # Bulgaria
    'EOR': 'Refugee Olympic Team',
    'PUR': 'PRI',   # Puerto Rico
    'BRN': 'BHR'  # Bahrain
}

# Applying the manual mapping to a new 'iso3_code' column
medals_per_country['iso3_code'] = medals_per_country['country_code'].replace(manual_mapping)

medals_per_country = medals_per_country[(medals_per_country['iso3_code'] != 'Individual Neutral Athletes') & (medals_per_country['iso3_code'] != 'Refugee Olympic Team')]

# Merging the df containing medal count per country with the one containing each country's population
medals_total = medals_per_country.merge(world_population, left_on = 'iso3_code', right_on = 'ISO3 Alpha-code', how = 'left')

# Converting the population of each country to millions
medals_total['Population (mn)'] = medals_total['Population (thousands)'].div(1000).astype(float).round(2)

# Rounding values to two decimal places
medals_total[['Population Sex Ratio (males per 100 females)', 'Median Age']] = medals_total[['Population Sex Ratio (males per 100 females)', 'Median Age']].round(2)

# Dropping redundant columns
medals_total.drop(columns = ['country_code', 'iso3_code', 'ISO2 Alpha-code', 'Type', 'Population (thousands)'], inplace = True)

medals_total['Country'].replace('United States of America', 'United States', inplace=True)

# Removing countries for which obtaining HDI is challenging or not trustworthy
countries_to_remove = ['China, Hong Kong SAR', 'China, Taiwan Province of China', "Dem. People's Republic of Korea", 'Kosovo (under UNSC res. 1244)', 'Puerto Rico']
medals_total = medals_total[~medals_total['Country'].isin(countries_to_remove)]

medals_total.head()

## Keeping only the countries which have won a medal at the Paris Olympics 2024

In [ ]:
# All countries that won a medal
medalist_countries = medals_total['Country'].unique()
medalist_countries.sort()
print("Number of countries that won a medal at the Paris Olympics 2024: ", len(medalist_countries))

all_hdi_countries = hdi['Country'].unique()
all_medalists_hdi = [country for country in all_hdi_countries if country in medalist_countries]
print("Number of medalist nations for which HDI data is available: ", len(all_medalists_hdi))

## Adding the HDI data & Total Number of Athletes to the comprehensive dataframe 'medals_total'

In [ ]:
medals_total = medals_total.merge(hdi, left_on = 'Country', right_on = 'Country', how = 'left')
medals_total = medals_total.merge(athletes_per_country, left_on = 'Country', right_on = 'Country', how = 'left')
medals_total.head()

# Working with the complete database

## Random Forest

In [ ]:
# Feature selection
X = dataset[['Population (mn)', '# Athletes', 'HDI']]
y = dataset['Total']
countries = dataset['Country']

# Train-test split
X_train, X_test, y_train, y_test, countries_train, countries_test = train_test_split(
    X, y, countries, test_size=0.2, random_state=42)

# Model training - Random Forest
model = RandomForestRegressor(random_state=42)

# Parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Custom cross-validation with KFold
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Create a list of all parameter combinations
param_list = list(ParameterGrid(param_grid))

# Initialize tqdm progress bar
progress_bar = tqdm(total=len(param_list) * cv.get_n_splits(), desc="Grid Search Progress")

# Custom loop over parameter grid
best_score = -float('inf')
best_params = None

for params in param_list:
    model.set_params(**params)
    cv_scores = []

    for train_idx, val_idx in cv.split(X_train):
        model.fit(X_train.iloc[train_idx], y_train.iloc[train_idx])
        y_val_pred = model.predict(X_train.iloc[val_idx])
        score = r2_score(y_train.iloc[val_idx], y_val_pred)
        cv_scores.append(score)

        progress_bar.update(1)

    avg_score = sum(cv_scores) / len(cv_scores)

    if avg_score > best_score:
        best_score = avg_score
        best_params = params

progress_bar.close()

print(f'Best parameters: {best_params}')

# Training the best model with the full training data
best_model = RandomForestRegressor(**best_params, random_state=42)
best_model.fit(X_train, y_train)

# Prediction on the test set
y_pred = best_model.predict(X_test)

# Evaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

# Feature importances
importances = best_model.feature_importances_
feature_importances = pd.Series(importances, index=X.columns).sort_values(ascending=False)
print(f'Feature Importances:\n{feature_importances}')

# Creating a DataFrame to display the results
results_df = pd.DataFrame({
    'Country': countries_test,
    'Actual Medals': y_test,
    'Predicted Medals': y_pred
})

In [ ]:
results_df

## Gradient Boosting

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test, countries_train, countries_test = train_test_split(
    X, y, countries, test_size=0.2, random_state=42)

# Model training - Example with Gradient Boosting and GridSearchCV for hyperparameter tuning
model2 = GradientBoostingRegressor(random_state=42)

# Define the parameter grid for Gradient Boosting
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Custom cross-validation with KFold
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Create a list of all parameter combinations
param_list = list(ParameterGrid(param_grid))

# Initialize tqdm progress bar
progress_bar = tqdm(total=len(param_list) * cv.get_n_splits(), desc="Grid Search Progress")

# Custom loop over parameter grid
best_score = -float('inf')
best_params = None

for params in param_list:
    model2.set_params(**params)
    cv_scores = []

    for train_idx, val_idx in cv.split(X_train):
        model2.fit(X_train.iloc[train_idx], y_train.iloc[train_idx])
        y_val_pred = model2.predict(X_train.iloc[val_idx])
        score = r2_score(y_train.iloc[val_idx], y_val_pred)
        cv_scores.append(score)

        progress_bar.update(1)

    avg_score = sum(cv_scores) / len(cv_scores)

    if avg_score > best_score:
        best_score = avg_score
        best_params = params

progress_bar.close()

print(f'Best parameters: {best_params}')

# Train the best model with the full training data
best_model = GradientBoostingRegressor(**best_params, random_state=42)
best_model.fit(X_train, y_train)

# Prediction on the test set
y_pred = best_model.predict(X_test)

# Evaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

# Feature importance
importances_model2 = best_model.feature_importances_
feature_importances_model2 = pd.Series(importances_model2, index=X.columns).sort_values(ascending=False)
print(f'Feature Importances:\n{feature_importances_model2}')

# Create a DataFrame to display the results
model_2_results = pd.DataFrame({
    'Country': countries_test,
    'Actual Medals': y_test,
    'Predicted Medals': y_pred
})

# Plotting Predicted vs Actual Medals
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.7, color='b')  # Scatter plot of actual vs predicted
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=2)  # Diagonal line (ideal case)
plt.xlabel('Actual Medals')
plt.ylabel('Predicted Medals')
plt.title('Predicted vs. Actual Medals')
plt.grid(True)
plt.show()

## Applying the best GradientBoosting Regressor to the entire dataset

In [ ]:
best_model = GradientBoostingRegressor(**best_params, random_state=42)
best_model.fit(X, y)

# Prediction on the entire dataset
y_pred = best_model.predict(X).round(0).astype(int)

# Evaluation
mse = mean_squared_error(y, y_pred)
r2 = r2_score(y, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

# Feature importance
importances_2_2 = best_model.feature_importances_
feature_importances_2_2 = pd.Series(importances_2_2, index=X.columns).sort_values(ascending=False)
print(f'Feature Importances:\n{feature_importances_2_2}')

# Create a DataFrame to display the results
results_2_2 = pd.DataFrame({
    'Country': countries,
    'Actual Medals': y,
    'Predicted Medals': y_pred
})

In [ ]:
results_2_2.sort_values(by = 'Actual Medals', ascending = False)

# Historical Olympics Data

In [ ]:
athletes_since_2000 = pd.read_csv('/content/Olympics 2000 - 2016 - athletes.csv')
noc_regions = pd.read_csv('/content/NOC Regions.csv')

# Adding the NOC regions to the athletes dataframe
athletes_since_2000 = pd.merge(athletes_since_2000, noc_regions, on='NOC', how='left')

# Calculating the participants from each country and each edition of the Olympics
athletes_per_country_per_year = athletes_since_2000.groupby(['Year', 'region'])['Name'].nunique().reset_index()
athletes_per_country_per_year.rename(columns = {'Name': '# Athletes', 'region' : 'Country'}, inplace = True)

# Retrieving the medalists from each competing country at each edition of the Olympics
medalists_per_country_per_year = athletes_since_2000[athletes_since_2000['Medal'].notna()].groupby(['Year', 'region', 'Sport', 'Event', 'Medal'])['Name'].nunique().reset_index()
medalists_per_country_per_year = medalists_per_country_per_year.groupby(['Year', 'region'])['Name'].count().reset_index()
medalists_per_country_per_year.rename(columns = {'Name': '# Medals', 'region' : 'Country'}, inplace = True)

# Merging the dataframes
merged_df = pd.merge(athletes_per_country_per_year, medalists_per_country_per_year, on = ['Year', 'Country'], how = 'left')
merged_df['# Medals'] = merged_df['# Medals'].fillna(0).astype(int)

################################################################# Adding the Tokyo Olympics #################################################################

# The medals for each country at the Tokyo Olympics
medals_tokyo = pd.read_excel('/content/Tokyo Olympics - Medals.xlsx')
medals_tokyo.rename(columns = {'Team/NOC': 'Country'}, inplace = True)
medals_tokyo['Year'] = 2021

# The athletes from each country at the Tokyo Olympics
athletes_tokyo = pd.read_excel('/content/Tokyo Olympics - Athletes.xlsx')
athletes_tokyo['Year'] = 2021
print(len(medals_tokyo))

# Total number of athletes per country at the Tokyo Olympics
tokyo_athletes_per_country = athletes_tokyo.groupby('NOC').size().reset_index(name = '# Athletes')
tokyo_athletes_per_country.rename(columns = {'NOC': 'Country'}, inplace = True)

# Merging the two dataframes
medals_tokyo = pd.merge(medals_tokyo, tokyo_athletes_per_country, on = 'Country', how = 'left')

# Creating a new dataframe from 'tokyo_df' to match the structure of 'merged_df'
tokyo_for_merge_df = medals_tokyo[['Year', 'Country', '# Athletes', 'Total']].copy()
tokyo_for_merge_df.rename(columns = {'Total': '# Medals'}, inplace = True)

# Concatenating the dataframes
merged_df = pd.concat([merged_df, tokyo_for_merge_df], ignore_index = True)

merged_df['Country'].replace(
    {'Macedonia': 'North Macedonia',
     'Iran (Islamic Republic of)' : 'Iran',
     "Côte d'Ivoire" : "Ivory Coast",
     'Republic of Moldova' : 'Moldova',
     'Syrian Arab Republic' : 'Syria'},
     inplace = True
    )

################################################################# Adding the Paris Olympics #################################################################
paris_olympics = pd.read_excel('/content/Paris Olympics Complete Dataset.xlsx')
paris_olympics['Year'] = paris_olympics['Year'].fillna(0).astype(int)
merged_df = pd.concat([merged_df, paris_olympics[['Year', 'Country', '# Athletes', '# Medals']]], ignore_index = True)

merged_df['Country'].replace(
    {'Macedonia': 'North Macedonia',
     'Iran (Islamic Republic of)' : 'Iran',
     "Côte d'Ivoire" : "Ivory Coast",
     'Republic of Moldova' : 'Moldova',
     'Syrian Arab Republic' : 'Syria',
     'Czechia' : 'Czech Republic',
     'Turkey' : 'Türkiye',
     'USA' : 'United States'},
     inplace = True
    )

################################################################# Adding Population Data #################################################################
country_population_per_year = pd.read_excel('/content/Population Data per Year per Country.xlsx')

# Selecting columns to convert (excluding the first one)
columns_to_convert = country_population_per_year.columns[1:]

# Converting population values to millions
country_population_per_year[columns_to_convert] = (country_population_per_year[columns_to_convert] / 1000000).round(2)

# Reshape the population dataframe from wide to long format
population_long_df = country_population_per_year.melt(id_vars = 'Year', var_name = 'Country', value_name = 'Population (mn)')

# Merge the reshaped population dataframe with the medals dataframe
merged_df = pd.merge(merged_df, population_long_df, on=['Year', 'Country'], how='left')

merged_df.sort_values(by = ['# Medals', 'Year'], ascending = False).head()

91


,Year,Country,# Athletes,# Medals,Population (mn)
1107,2024,United States,593,126,334.91
989,2016,United States,555,121,323.07
1003,2021,United States,615,113,332.05
584,2008,United States,588,110,304.09
786,2012,United States,529,103,313.88


# Olympics Complete Historical Dataset

In [ ]:
# Data Loading & EDA
olympics_data = pd.read_excel('/content/Olympics Complete Historical Database.xlsx')
print(olympics_data['Country'].nunique(), "unique countries have participated between Sydney 2000 and Paris 2024")

# Generating a report of the data
profile = ProfileReport(olympics_data, title = "Olympics Historical Data")

# Saving the report to .html for inspection
profile.to_file("olympics_report.html")

199 unique countries have participated between Sydney 2000 and Paris 2024


/usr/local/lib/python3.10/dist-packages/ydata_profiling/profile_report.py:363: UserWarning: Try running command: 'pip install --upgrade Pillow' to avoid ValueError
  warnings.warn(


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
hdi = pd.read_excel('/content/HDI 2023.xlsx')
hdi.columns = hdi.columns.str.strip()
hdi['Country'].replace({'Korea (Republic of)': 'Republic of Korea', 'Moldova (Republic of)': 'Republic of Moldova'}, inplace = True)

# Calculate the total number of medalists per country and sort them in descending order
top_countries_sorted = olympics_data.groupby('Country')['# Medals'].sum().sort_values(ascending = False).index[:10]
filtered_df = olympics_data[olympics_data['Country'].isin(top_countries_sorted)]

# Olympic Years
olympic_years = [2000, 2004, 2008, 2012, 2016, 2021, 2024]

color_map = px.colors.qualitative.T10
fig = go.Figure()

# Adding traces for each country
for i, country in enumerate(top_countries_sorted):
    country_data = filtered_df[filtered_df['Country'] == country].sort_values(by='Year')
    fig.add_trace(go.Scatter(x = country_data['Year'], y = country_data['# Medals'],
                             mode = 'lines+markers',
                             name = country,
                             line = dict(color = color_map[i], width = 3),
                             marker = dict(size = 8, symbol = 'circle'),
                             hovertemplate = f'<b>{country}</b><br>Medals: %{{y}}<extra></extra>'))

# Updating the layout for a more professional look
fig.update_layout(
    title = dict(text = 'Top 10 Countries by Number of Medals Over the Years', font = dict(size = 24, family = 'Arial', color = 'black')),
    xaxis_title = dict(text = 'Year', font = dict(size = 18, family = 'Arial')),
    yaxis_title = dict(text = 'Number of Medals', font = dict(size = 18, family = 'Arial')),
    xaxis = dict(tickmode = 'array', tickvals = olympic_years, ticktext = olympic_years, tickfont = dict(size = 14)),
    yaxis = dict(tickfont = dict(size = 14)),
    legend = dict(font = dict(size = 14), title_font_family = 'Arial', x = 1.05, y = 1),
    margin = dict(l = 50, r = 50, t = 100, b = 50),
    template = 'plotly_white',
    hovermode = 'x unified',
    height = 600,  # Make it more compact and readable
    width = 1000,  # Adjust width for better embed display
)

# Adding grid lines
fig.update_xaxes(showgrid = True, gridwidth = 0.5, gridcolor = 'lightgrey')
fig.update_yaxes(showgrid = True, gridwidth = 0.5, gridcolor = 'lightgrey')

# Disabling the range slider
fig.update_layout(xaxis_rangeslider_visible = False)

fig.show()

fig.write_html("top_10_countries_medals.html")

In [ ]:
olympics_data[olympics_data['Country'] == 'China']

,Year,Country,# Athletes,# Medals,Population (mn)
0,2000,China,297,58,1262.64
185,2004,China,404,64,1296.08
372,2008,China,617,100,1324.66
561,2012,China,406,89,1354.19
751,2016,China,427,70,1387.79
943,2021,China,401,88,1412.36
1035,2024,China,388,91,1410.71


## Predicting Medal Count at the Paris Olympics 2024

In [ ]:
366-327

39

In [ ]:
# Features & Target Variable
train_data = olympics_data[olympics_data['Year'] <= 2021]
X_train = train_data[['# Athletes', 'Population (mn)']]
y_train = train_data['# Medals']

test_data = olympics_data[olympics_data['Year'] == 2024]
X_test = test_data[['# Athletes', 'Population (mn)']]
y_test = test_data['# Medals']

In [ ]:
# Models Initialization
rf_model = RandomForestRegressor(random_state = 42)
xgb_model = XGBRegressor(random_state=  42)

# Models Training
rf_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)

# Models Predictions
rf_predictions = rf_model.predict(X_test)
xgb_predictions = xgb_model.predict(X_test)

# Ensuring that total predicted medals sum to 1010
rf_total_predicted = rf_predictions.sum()
rf_adjustment_factor = 1010 / rf_total_predicted
adjusted_rf_predictions = (rf_predictions * rf_adjustment_factor).astype(int)

xgb_total_predicted = xgb_predictions.sum()
xgb_adjustment_factor = 1010 / xgb_total_predicted
adjusted_xgb_predictions = (xgb_predictions * xgb_adjustment_factor).astype(int)

# Evaluating the models with adjusted predictions
rf_r2 = r2_score(y_test, adjusted_rf_predictions).round(2)
rf_mse = mean_squared_error(y_test, adjusted_rf_predictions).round(2)
print(f"Random Forest R²: {rf_r2}, MSE: {rf_mse}")

xgb_r2 = r2_score(y_test, adjusted_xgb_predictions).round(2)
xgb_mse = mean_squared_error(y_test, adjusted_xgb_predictions).round(2)
print(f"XGBoost R²: {xgb_r2}, MSE: {xgb_mse}")

# Feature Importances
rf_importances = rf_model.feature_importances_.round(2)
xgb_importances = xgb_model.feature_importances_.round(2)

# DataFrame containing the predictions
results_df = test_data[['Country']].copy()
results_df['Actual Medals'] = y_test.values
results_df['RF Predicted Medals'] = adjusted_rf_predictions
results_df['XGB Predicted Medals'] = adjusted_xgb_predictions
results_df['Year'] = 2024

# Print feature importances
feature_names = ['# Athletes', 'Population (mn)']
rf_importances_df = pd.DataFrame({'Feature': feature_names, 'Importance': rf_importances})
xgb_importances_df = pd.DataFrame({'Feature': feature_names, 'Importance': xgb_importances})

print("\nRandom Forest Feature Importances:")
print(rf_importances_df)

print("\nXGBoost Feature Importances:")
print(xgb_importances_df)

results_df = pd.merge(results_df, olympics_data[['Country', 'Population (mn)', '# Athletes', '# Medals', 'Year']], on = ['Country', 'Year'], how = 'left')
results_df = pd.merge(results_df, hdi, on = 'Country', how = 'left')

results_df = results_df[['Country', '# Athletes', '# Medals', 'RF Predicted Medals', 'XGB Predicted Medals', 'Population (mn)', 'HDI', 'HDI Bracket']]
results_df['# Medals Rank'] = results_df['# Medals'].rank(ascending = False, method = 'min').astype(int)
results_df['# RF Predicted Medals Rank'] = results_df['RF Predicted Medals'].rank(ascending = False, method = 'min').astype(int)
results_df['# XGB Predicted Medals Rank'] = results_df['XGB Predicted Medals'].rank(ascending = False, method = 'min').astype(int)

results_df.to_excel('Model Predictions.xlsx', index = False)

results_df.sort_values(by = 'XGB Predicted Medals', ascending = False)

Random Forest R²: 0.92, MSE: 32.22
XGBoost R²: 0.91, MSE: 36.8

Random Forest Feature Importances:
           Feature  Importance
0       # Athletes        0.82
1  Population (mn)        0.18

XGBoost Feature Importances:
           Feature  Importance
0       # Athletes        0.81
1  Population (mn)        0.19


,Country,# Athletes,# Medals,RF Predicted Medals,XGB Predicted Medals,Population (mn),HDI,HDI Bracket,# Medals Rank,# RF Predicted Medals Rank,# XGB Predicted Medals Rank
2,United States,593,126,126,132,334.91,0.927,Very High,1,1,1
1,China,388,91,93,98,1410.71,0.788,High,2,2,2
16,France,572,64,67,73,68.17,0.910,Very High,4,3,3
13,Germany,427,33,56,60,84.48,0.950,Very High,9,4,4
8,Japan,410,45,47,53,124.52,0.920,Very High,6,6,5
34,Australia,460,53,50,47,26.64,0.946,Very High,5,5,6
15,Great Britain,327,65,41,38,68.35,0.940,Very High,3,7,7
40,Netherlands,276,34,32,34,17.88,0.946,Very High,8,9,8
23,Spain,382,18,32,31,48.37,0.911,Very High,15,9,9
18,Italy,380,40,38,30,58.76,0.906,Very High,7,8,10


In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge

# Initialize the meta-model (stacking regressor)
# You can choose any regression model, here we're using Ridge Regression as an example
meta_model = Ridge()

# Create a stacking ensemble model
stacking_ensemble = StackingRegressor(
    estimators=[('rf', rf_model), ('xgb', xgb_model)],
    final_estimator=meta_model,
    passthrough=False  # If True, pass the original features along with the predictions to the meta-model
)

# Train the stacking ensemble model
stacking_ensemble.fit(X_train, y_train)

# Make predictions with the stacking ensemble model
stacking_predictions = stacking_ensemble.predict(X_test)

# Adjust the predictions to sum to 1010 medals
stacking_total_predicted = stacking_predictions.sum()
stacking_adjustment_factor = 1010 / stacking_total_predicted
adjusted_stacking_predictions = (stacking_predictions * stacking_adjustment_factor).astype(int)

# Evaluate the stacking ensemble model
stacking_r2 = r2_score(y_test, adjusted_stacking_predictions).round(2)
stacking_mse = mean_squared_error(y_test, adjusted_stacking_predictions).round(2)
print(f"Stacking Ensemble Model R²: {stacking_r2}, MSE: {stacking_mse}")

# Add predictions to the results DataFrame
results_df['Stacking Ensemble Predicted Medals'] = adjusted_stacking_predictions

# Update the results DataFrame with ensemble predictions
results_df['Voting Ensemble Predicted Medals Rank'] = results_df['Voting Ensemble Predicted Medals'].rank(ascending=False, method='min').astype(int)
results_df['Stacking Ensemble Predicted Medals Rank'] = results_df['Stacking Ensemble Predicted Medals'].rank(ascending=False, method='min').astype(int)

# Save the updated DataFrame to Excel
results_df.to_excel('Model Predictions with Ensembles.xlsx', index=False)

# Print and sort by the ensemble predictions
results_df.sort_values(by='Stacking Ensemble Predicted Medals', ascending=False)

Stacking Ensemble Model R²: 0.91, MSE: 37.61


,Country,# Athletes,# Medals,RF Predicted Medals,XGB Predicted Medals,Population (mn),HDI,HDI Bracket,# Medals Rank,# RF Predicted Medals Rank,# XGB Predicted Medals Rank,Voting Ensemble Predicted Medals,Stacking Ensemble Predicted Medals,Voting Ensemble Predicted Medals Rank,Stacking Ensemble Predicted Medals Rank
2,United States,593,126,126,132,334.91,0.927,Very High,1,1,1,129,129,1,1
1,China,388,91,91,98,1410.71,0.788,High,2,2,2,94,93,2,2
16,France,572,64,65,73,68.17,0.910,Very High,4,3,3,69,64,3,3
13,Germany,427,33,58,60,84.48,0.950,Very High,9,4,4,59,59,4,4
34,Australia,460,53,53,47,26.64,0.946,Very High,5,5,6,50,57,5,5
8,Japan,410,45,46,53,124.52,0.920,Very High,6,6,5,50,46,5,6
18,Italy,380,40,39,30,58.76,0.906,Very High,7,8,10,34,43,8,7
15,Great Britain,327,65,41,38,68.35,0.940,Very High,3,7,7,40,43,7,7
23,Spain,382,18,33,31,48.37,0.911,Very High,15,9,9,32,35,10,9
40,Netherlands,276,34,33,34,17.88,0.946,Very High,8,9,8,34,34,8,10


In [ ]:
outperformed = results_df[(results_df['# Medals'] > results_df['RF Predicted Medals'])&(results_df['# Medals'] > results_df['XGB Predicted Medals'])]
print(f"Number of countries that won more medals than expected: {len(outperformed)}")
outperformed.sort_values(by = '# Medals', ascending = False).head(15)

Number of countries that won more medals than expected: 37


,Country,# Athletes,# Medals,RF Predicted Medals,XGB Predicted Medals,Population (mn),HDI,HDI Bracket,# Medals Rank,# RF Predicted Medals Rank,# XGB Predicted Medals Rank
15,Great Britain,327,65,41,38,68.35,0.940,Very High,3,7,7
18,Italy,380,40,39,30,58.76,0.906,Very High,7,8,10
21,South Korea,141,32,8,12,51.71,0.929,Very High,10,23,16
26,Canada,316,27,19,19,40.10,0.935,Very High,11,13,14
54,Hungary,169,19,17,16,9.59,0.851,Very High,14,15,15
30,Uzbekistan,86,13,5,4,36.41,0.727,High,16,43,41
28,Ukraine,140,12,7,8,37.00,0.734,High,17,30,26
11,Iran,39,12,6,5,89.17,0.780,High,17,34,37
19,Kenya,70,11,8,10,55.10,0.601,Medium,19,23,21
46,Cuba,61,9,5,4,11.19,0.764,High,23,43,41


## Ensemble Model

In [ ]:
print("Random Forest Parameters:\n", rf_model.get_params())
print("XGBoost Parameters:\n", xgb_model.get_params())

Random Forest Parameters:
 {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}
XGBoost Parameters:
 {'objective': 'reg:squarederror', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'feature_types': None, 'gamma': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': None, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': None, 'max_leaves': None, 'min_child_weight': None, 'missing': 

In [ ]:
# Generate predictions using cross-validation for each base model
preds_rf = cross_val_predict(RandomForestRegressor(n_estimators=100, random_state=42), X_train, y_train, cv=5)
preds_xgb = cross_val_predict(XGBRegressor(n_estimators=100, random_state=42), X_train, y_train, cv=5)

# Combine predictions into a DataFrame
preds_df = pd.DataFrame({
    'RandomForest': preds_rf,
    'XGBoost': preds_xgb
})

# Calculate the correlation matrix
correlation_matrix = preds_df.corr()
correlation_matrix

,RandomForest,XGBoost
RandomForest,1.000000,0.970837
XGBoost,0.970837,1.000000
